# QUANTITATIVE VALUE METRICS

value investing - investing in stocks that are the cheapest relative to common measures of business ie: earnings / assets

# PRICE TO EARNINGS RATIO - stock price / Yearly Earnings Forcast

always strictly define your P/E Ratio because there are many different metrics on finding this ratio... 

In [35]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

Importing Stocks and API token

In [36]:
# SMP 500 CSV
stocks = pd.read_csv("sp_500_stocks.csv")

# api
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

# SORTING ALL STOCKS INTO 2D ARRAY HOLDING A LIST OF 100 STOCKS EACH
[
    [100],
    [100],
    [100],
    [100],
    [100],
]

# TURN EACH LIST INTO A STRING 
[
    'X' * 100 ,
    'X' * 100 ,
    'X' * 100 ,
    'X' * 100 ,
    'X' * 100 ,
]

In [37]:
# function to split bigger list into smaller lists
def chunks( lst , n  ):
    ''' yeild successive n-sized lists from lst'''
    for i in range( 0 , len(lst) , n ):
        yield lst[i: i + n]

# 2 dimentional list with each inner list containing 100 tickers
symbol_groups = list(chunks( stocks['Ticker'] , 100  ) )
symbol_strings = []

# turns each list into usable string for api call
for i in range( 0 , len(symbol_groups) , 1  ):
    joined_arr =  ",".join(symbol_groups[i])
    symbol_strings.append( joined_arr )


CREATING FINAL OUTPUT DATAFRAME

In [38]:
# creating columns for dataframe
my_columns = [
    'Ticker',
    'Price',
    'P/E Ratio',
    'Number of Shares To Buy',
]

final_dataframe = pd.DataFrame(columns = my_columns)

# BATCH API CALLS to get market data

Also populates final_dataframe

In [39]:
# create url for each list of 100 stocks
# request data for the sub list
for group_string in symbol_strings:
    batch_api_call = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={group_string}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get( batch_api_call ).json()

    # arr of every ticker in sub list
    stock_array_in_sub_list = group_string.split(',')
    for symbol in stock_array_in_sub_list:
        price = data[symbol]['quote']['latestPrice']
        pe_ratio = data[symbol]['quote']['peRatio']
        
        #NOTE ***** append to final_dataframe *****
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    price,
                    pe_ratio,
                    "N/A"
                ],
                index = my_columns
            ),
            ignore_index = True
        )


# final_dataframe

# Selecting top 50 stocks

In [40]:
# Sorting values in DECENDING order and saving it for later use using INPLACE = TRUE
final_dataframe.sort_values( "P/E Ratio" , ascending = False , inplace = True   )

# Remove all stocks that have negative P/E ratios (they lost money this year)
losers = final_dataframe[final_dataframe["P/E Ratio"]  < 0 ]
final_dataframe =  final_dataframe[final_dataframe["P/E Ratio"]  > 0 ]

# save final_dataframe[:50] as original 
# final_dataframe = final_dataframe[:50]

# Resetting index of sorted data_frame
final_dataframe.reset_index(  inplace = True , drop = True )

# # removing Old Index Column
# final_dataframe.drop(labels='index', axis= 'columns', inplace=True, errors='raise')

# final_dataframe
# losers

CALCULATING NUMBER OF SHARES TO BUY

In [42]:
# validate user input for portfollio size '$$$'
def user_portfollio_size_input(portfollio_size):
    try:
        val = float(portfollio_size)
        print(f"Your have entered: ${val}\n")
        return True
    except ValueError:
        print("ERROR: Please type a numerical value")
        return False

portfollio_size = input("Please enter value of your portfollio: $")
while not user_portfollio_size_input(portfollio_size):
    portfollio_size = input("Please enter value of your portfollio: $")

# calculate position size
portfollio_size = float(portfollio_size)
position_size = float(portfollio_size) / len(final_dataframe.index)
print(f"Your position size is ${'{:,.2f}'.format(position_size)}")

Your have entered: $10000.0

Your position size is $21.46


In [43]:
# update final data frame
for x in range(0 , len(final_dataframe) , 1):
    final_dataframe.loc[x , "Number of Shares To Buy"] =  position_size /  final_dataframe.loc[x , "Price"]


final_dataframe

,Ticker,Price,P/E Ratio,Number of Shares To Buy
0,LUV,42.650,13065.7,0.503147
1,HLT,160.500,1166.25,0.133702
2,NOW,681.030,631.88,0.03151
3,ALK,53.280,513.45,0.402763
4,UDR,60.930,331.94,0.352195
...,...,...,...,...
461,HPQ,38.250,3.68,0.561026
462,COO,435.481,3.68,0.0492771
463,EBAY,66.410,3.63,0.323132
464,BIO,765.040,3.62,0.0280498
